In [6]:
import glob
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter


In [7]:
# get files
files = []
for file in glob.glob('data/workshop-content18/4-comm100/data/comm100_data/*'):
    f = open(file, "r", encoding="utf-16-le")
    fContent = f.read()[1:]
    files.append(fContent)
    f.close

#print(len(files))
# files[0]


In [8]:
# parse each file as a list of (conversation author + conversation content)
conversations = [] # for all files
for file in files :
    conversation = [] # for all conversation rounds in each file
    lines = file.split("\n")
    for line in lines:
        #print(line)
        if re.match(r'^\s*$', line): continue # reject blank lines
        
        author = []
        content = []
        if ':' in line == -1: # no author is found
            if not conversation: continue # file starts with nobody saying something - nonsense
            else: author = conversation[-1][0] # inherit the previous author if no author is found
            content = line
        else:
            author = line.split(":")[0]
            content = ' '.join(line.split(":")[1:])
            
        if re.match(r'^\s*$', author) or re.match(r'^\s*$', content): continue
                  
        conversationSingle = []
        conversationSingle.append(author)
        conversationSingle.append(content)
        conversation.append(conversationSingle)

    conversations.append(conversation) 
            

In [9]:
# # preprocess the conversations to reject trivial sentences
# for convIdx, conversation in enumerate(conversations):
#     for sentIdx, sentence in enumerate(conversation):
#         author = sentence[0]
#         content = sentence[1]
#         if author.find("Agent") != -1: author = "agent"
#         else: author = "client"
#         sentence[0] = author
        
#         # do something:



In [25]:
# word frequency stats
words = []
nltk.download('stopwords')
stop = stopwords.words('english')
for conversation in conversations:
    contents = [x[1] for x in conversation]
    contentCombined = ' '.join(contents)
    noPuncConv = re.sub('[' + punctuation + ']',' ', contentCombined)
    words.append([word.lower() for word in noPuncConv.split() if word.lower() not in stop])
  
# frequency for all conversation
wordCombined = []
for w in words:
    wordCombined += w
frequency_ = Counter(wordCombined).most_common()   
print(frequency_)  


# frequency for each conversation
frequency = []
for w in words:
    frequency = Counter(w).most_common()
    print(frequency)
    
# # optional
# # merge consecutive sentences with the same author to a single sentence
# for conversation in conversations:
#     authors = [x[0] for x in conversation]
#     authors_ = [1 if 'Agent' in x else 0 for x in authors]
#     indices = [i for i, s in enumerate(authors) if 'Agent' in s]
#     print(authors)
#     print(indices)
#     #authors[indices] = 1
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/xcodist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[('chat', 16700), ('comm100', 14254), ('help', 9685), ('hi', 9073), ('may', 6157), ('please', 6088), ('today', 5624), ('ok', 5608), ('com', 5111), ('email', 4629), ('agent', 4508), ('hello', 4497), ('yes', 4468), ('thank', 4412), ('left', 4411), ('visitor', 4358), ('live', 3987), ('account', 3768), ('thanks', 3523), ('terry', 3450), ('website', 3420), ('need', 2998), ('one', 2902), ('get', 2884), ('check', 2790), ('norman', 2680), ('us', 2669), ('sent', 2493), ('ticket', 2447), ('see', 2415), ('brian', 2312), ('know', 2304), ('www', 2294), ('let', 2286), ('want', 2246), ('sure', 2109), ('time', 1978), ('like', 1937), ('file', 1898), ('team', 1883), ('would', 1867), ('transcript', 1840), ('contacting', 1796), ('https', 1715), ('problem', 1708), ('day', 1632), ('bye', 1630), ('system', 1628), ('use', 1615), ('send', 1550), ('operator', 1536), ('so

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
myData = pd.DataFrame(conversations)

In [8]:
myData.head()

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,"[agent, Hello Support 03, this is Brian. How ...","[client, one of my site is http //vibong88.co...","[client, i have change ""Greeting Messag"", but...","[agent, When you say "" greeting message"" , yo...","[client, ok]","[agent, Can you please double check if you ar...","[client, yes it right. I have changed same fo...","[agent, Please hold on, let me see. ]","[agent, I am on this website http //vibong8...","[client, yes]",...,None,None,None,None,None,None,None,None,None,None
1,"[client, Toyota hybrid 2014 looking]","[agent, Dear khalid, thank you for contacting...","[client, I am looking Toyota hybrid from 2013]","[agent, Hi, I’m trying to help but you might ...","[client, The visitor has left the chat.]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"[agent, Hi Max, this is Terry with Comm100. H...","[client, Hi Terry, I have a question on the C...","[agent, Ok]","[agent, go ahead]","[client, I have a website - a responsive. whe...","[agent, Currently for all the mobile site, th...","[client, How do i override the default settin...","[agent, The will require an additional custom...","[client, Oh!, But the same works as expected ...","[agent, Yes, Embedded Chat window does not wo...",...,None,None,None,None,None,None,None,None,None,None
3,"[agent, Hello Kelzang Drakpa, this is Brian. ...","[client, Yes]","[agent, How can I help you?]","[client, Please help me prepare for IELTS tes...","[agent, Hi, I’m trying to help but you might ...","[client, Thank you sir, how can i contact wit...","[agent, sorry but you are wrongly directed]","[client, The visitor has left the chat.]",None,None,...,None,None,None,None,None,None,None,None,None,None
4,"[agent, Hello Ragul, this is Norman with Comm...","[client, Hi]","[client, Hi]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
